In [ ]:
# 任务类，包含任务内容和优先级  
class EventTask:  
    def __init__(self, priority, event):  
        self.priority = priority  
        self.event = event  
  
    # 定义小于比较方法，用于优先级队列排序 , 
    def __lt__(self, other):  
        return self.priority > other.priority

class EventEngine(object):


    #----------------------------------------------------------------------
    def __init__(self):
        """初始化事件引擎"""
        # 事件队列
        self.priority_queue_ = queue.PriorityQueue()
        
        # 事件处理线程
        self.task_thread_ = threading.Thread(target = self.run_)
        
        # 计时器，用于触发计时器事件
        self.timer_thread_ = threading.Thread(target = self.runTimer_)
        self.timerSleep_= 1                           # 计时器触发间隔（默认1秒）        
        # 停止信号
        self.stop_ = True
        # 这里的handlers_是一个字典，用来保存对应的事件调用关系
        self.handlers_ = {}
        self.lock = threading.Lock() 
        
    #----------------------------------------------------------------------
    def run_(self):
        """运行"""
        while self.stop_ == False:
            if not self.priority_queue_.empty():
                # 获取事件的阻塞时间设为1秒
                event_task = self.priority_queue_.get(block = True, timeout = 1)  
                self.process_(event_task.event)
            
    #----------------------------------------------------------------------
    def process_(self, event):
        """处理事件"""
        # 检查是否存在对该事件进行监听的处理函数
        if event.type in self.handlers_:
            # 若存在，则按顺序将事件传递给处理函数执行
#             print("self.handlers_", self.handlers_)
            for handler in self.handlers_[event.type]:
                handler(event)    
               
    #----------------------------------------------------------------------
    def runTimer_(self):
        """运行在计时器线程中的循环函数"""
        while not self.stop_ :
            # 创建计时器事件
            event = TimerEvent()  
            task = EventTask(1, event)
       
            # 向队列中存入计时器事件
            self.put(task)    
             
            # 等待
            time.sleep(self.timerSleep_)

    #----------------------------------------------------------------------
    def start(self):
        """引擎启动"""
        # 将引擎设为启动
        self.stop_ = False
        
        # 启动事件处理线程
        self.task_thread_.start()
        
        # 启动计时器，计时器事件间隔默认设定为1秒
        self.timer_thread_.start()
    
    #----------------------------------------------------------------------
    def stop(self):
        """停止引擎"""
        # 停止
        self.stop_ = True
        
        # 计时器
        self.timer_thread_.join()
        
        # 等待事件处理线程退出
        self.task_thread_.join()
            
    #----------------------------------------------------------------------
    def register(self, type_name, handler):
        """注册事件处理函数监听"""
        # 尝试获取该事件类型对应的处理函数列表，若无则创建
        try:
            handlerList = self.handlers_[type_name]
        except KeyError:
            handlerList = []
            self.handlers_[type_name] = handlerList
        
        # 若要注册的处理器不在该事件的处理器列表中，则注册该事件
        if handler not in handlerList:
            handlerList.append(handler)
            
    #----------------------------------------------------------------------
    def unregister(self, type_name, handler):
        """注销事件处理函数监听"""
        # 尝试获取该事件类型对应的处理函数列表，若无则忽略该次注销请求
        try:
            handlerList = self.handlers_[type_name]
#             print("find type_name:", type_name)
            # 如果该函数存在于列表中，则移除
            if handler in handlerList:
#                 print("remove handler")
#                 print("handlerList1", handlerList)
                handlerList.remove(handler)
#                 print("handlerList2", handlerList)

            # 如果函数列表为空，则从引擎中移除该事件类型
            if not handlerList:
                del self.handlers_[type_name]
        except KeyError:
            pass     
        
    #----------------------------------------------------------------------
    def put(self, priority_event):
        """向事件队列中存入事件"""
        
        self.priority_queue_.put(priority_event)

# 处理时间信号
def handle_timer_event(event):
    print(f"recv timer event:{event.type}")

# 处理行情信号    
def handle_quote_event(event):
    print(f"recv quote event:{event.type}")

# 添加任务到优先级队列  
def add_quote_tasks(event_engine,start, end, step):  
    for i in range(start, end, step):  
        priority = random.randint(1, 10)  # 随机生成优先级  
        content = QuoteEvent()  
        task = EventTask(priority, content)
        event_engine.put(task) 
        print(f"添加任务: quote, 优先级: {priority}")  